In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '/home/federico/Università/Tesi_magistrale/my_programs/')

from contact_matrix_utils import *
#from src.interchromosomal_cosegregation import *
from numpy.random import Generator, MT19937
from scipy.stats import pearsonr, spearmanr
from scipy.spatial.distance import squareform
from src.utilities import *
import warnings

import seaborn as sns

In [2]:
save_folder = "/home/federico/Università/Tesi_magistrale/images/1_chr12_chr18_dataset_comparison.ipynb_output/"

In [3]:
def select_chromosomes_coordinates(chrA, chrB, chr_windows_sizes):
    if(chrA==0 or chrB ==0):
        print("0 is not accepted as chromosome index ")
        return

    chrA_idx, chrB_idx = chrA - 1, chrB - 1

    if(chrA_idx>chrB_idx):
        chrA_idx, chrB_idx = chrB_idx, chrA_idx

    start_row    = np.sum(chr_windows_sizes[:chrA_idx])
    stop_row     = start_row + chr_windows_sizes[chrA_idx]
    start_column = np.sum(chr_windows_sizes[:chrB_idx])
    stop_column  = start_column + chr_windows_sizes[chrB_idx]

    return start_row, stop_row, start_column, stop_column
which_beta = "$\\beta$ evaluated | 1/2"

# Import coseg #

In [4]:
name_root_1NP_150kb = "mesc_46C_150kb_481x1"
name_root_iza_150kb = "iza-mesc_150kb_420x3"
name_root_dopa_150kb = "dopa30_150kb_482x3"

path_1 =      r"/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/" + name_root_1NP_150kb + "/"
file_name_1 = "segregation_" + name_root_1NP_150kb +  ".pkl"
with open(path_1 + file_name_1, 'rb') as f:
    segregation_table_1NP_150kb = pickle.load(f)

path_2 =      r"/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/" + name_root_iza_150kb + "/"
file_name_2 = "segregation_" + name_root_iza_150kb +  ".pkl"
with open(path_2 + file_name_2, 'rb') as f:
    segregation_table_iza_150kb = pickle.load(f)

path_3 =      r"/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/" + name_root_dopa_150kb + "/"
file_name_3 = "segregation_" + name_root_dopa_150kb +  ".pkl"
with open(path_3 + file_name_3, 'rb') as f:
    segregation_table_dopa_150kb = pickle.load(f)

start_chr, stop_chr = 1, 19
stop_chr += 1
chromosomes = []
n_chromosomes = stop_chr - start_chr

l = []
size = 0

for i in range(start_chr, stop_chr):
    chrA = 'chr' + str(i)
    chromosomes += [chrA]
    s = segregation_table_1NP_150kb[chrA]['segregation_table'].shape[0]
    size += s
    l.append(s) 

chr_windows_sizes = np.array(l)

In [5]:
chr12_1NP_150kb_coseg = compute_tube_cosegregation_matrix(segregation_table_1NP_150kb['chr12']['segregation_table'])
chr18_1NP_150kb_coseg = compute_tube_cosegregation_matrix(segregation_table_1NP_150kb['chr18']['segregation_table'])
chr12_18_1NP_150kb_coseg = compute_tube_cosegregation_matrix_offdiag(segregation_table_1NP_150kb['chr12']['segregation_table'], segregation_table_1NP_150kb['chr18']['segregation_table'])

chr12_1NP_150kb_coseg[chr12_1NP_150kb_coseg == 0] = np.nan
chr18_1NP_150kb_coseg[chr18_1NP_150kb_coseg == 0] = np.nan
chr12_18_1NP_150kb_coseg[chr12_18_1NP_150kb_coseg == 0] = np.nan

chr12_iza_150kb_coseg = compute_tube_cosegregation_matrix(segregation_table_iza_150kb['chr12']['segregation_table'])
chr18_iza_150kb_coseg = compute_tube_cosegregation_matrix(segregation_table_iza_150kb['chr18']['segregation_table'])
chr12_18_iza_150kb_coseg = compute_tube_cosegregation_matrix_offdiag(segregation_table_iza_150kb['chr12']['segregation_table'], segregation_table_iza_150kb['chr18']['segregation_table'])

chr12_iza_150kb_coseg[chr12_iza_150kb_coseg == 0] = np.nan
chr18_iza_150kb_coseg[chr18_iza_150kb_coseg == 0] = np.nan
chr12_18_iza_150kb_coseg[chr12_18_iza_150kb_coseg == 0] = np.nan

chr12_dopa_150kb_coseg = compute_tube_cosegregation_matrix(segregation_table_dopa_150kb['chr12']['segregation_table'])
chr18_dopa_150kb_coseg = compute_tube_cosegregation_matrix(segregation_table_dopa_150kb['chr18']['segregation_table'])
chr12_18_dopa_150kb_coseg = compute_tube_cosegregation_matrix_offdiag(segregation_table_dopa_150kb['chr12']['segregation_table'], segregation_table_dopa_150kb['chr18']['segregation_table'])

chr12_dopa_150kb_coseg[chr12_dopa_150kb_coseg == 0] = np.nan
chr18_dopa_150kb_coseg[chr18_dopa_150kb_coseg == 0] = np.nan
chr12_18_dopa_150kb_coseg[chr12_18_dopa_150kb_coseg == 0] = np.nan

In [6]:
chr12_18_WDF_1NP_150kb = np.array([])

segregation_table_12 = segregation_table_1NP_150kb["chr12"]["segregation_table"]
chr12_WDF_1NP_150kb = compute_tube_segregation_frequency(segregation_table_12)

segregation_table_18 = segregation_table_1NP_150kb["chr18"]["segregation_table"]
chr18_WDF_1NP_150kb = compute_tube_segregation_frequency(segregation_table_18)

chr12_18_WDF_1NP_150kb = np.concatenate((chr12_WDF_1NP_150kb, chr18_WDF_1NP_150kb))

del segregation_table_12, segregation_table_18

chr12_18_WDF_iza_150kb = np.array([])

segregation_table_12 = segregation_table_iza_150kb["chr12"]["segregation_table"]
chr12_WDF_iza_150kb = compute_tube_segregation_frequency(segregation_table_12)

segregation_table_18 = segregation_table_iza_150kb["chr18"]["segregation_table"]
chr18_WDF_iza_150kb = compute_tube_segregation_frequency(segregation_table_18)

chr12_18_WDF_iza_150kb = np.concatenate((chr12_WDF_iza_150kb, chr18_WDF_iza_150kb))

del segregation_table_12, segregation_table_18

chr12_18_WDF_dopa_150kb = np.array([])

segregation_table_12 = segregation_table_dopa_150kb["chr12"]["segregation_table"]
chr12_WDF_dopa_150kb = compute_tube_segregation_frequency(segregation_table_12)

segregation_table_18 = segregation_table_dopa_150kb["chr18"]["segregation_table"]
chr18_WDF_dopa_150kb = compute_tube_segregation_frequency(segregation_table_18)

chr12_18_WDF_dopa_150kb = np.concatenate((chr12_WDF_dopa_150kb, chr18_WDF_dopa_150kb))

del segregation_table_12, segregation_table_18

# Import $\pi$ #

In [7]:
chr12_1NP_150kb_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/mesc_46C_150kb_481x1/PI2/PI2_chr12_mesc_46C_150kb_481x1.txt"))
chr12_iza_150kb_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/iza-mesc_150kb_420x3/PI2/PI2_chr12_iza-mesc_150kb_420x3.txt"))
chr12_dopa_150kb_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/dopa30_150kb_482x3/PI2/PI2_chr12_dopa30_150kb_482x3.txt"))

chr18_1NP_150kb_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/mesc_46C_150kb_481x1/PI2/PI2_chr18_mesc_46C_150kb_481x1.txt"))
chr18_iza_150kb_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/iza-mesc_150kb_420x3/PI2/PI2_chr18_iza-mesc_150kb_420x3.txt"))
chr18_dopa_150kb_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/dopa30_150kb_482x3/PI2/PI2_chr18_dopa30_150kb_482x3.txt"))

chr12_1NP_150kb_sign_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/mesc_46C_150kb_481x1/PI2/PI2_significant_95_chr12_mesc_46C_150kb_481x1.txt"))
chr12_iza_150kb_sign_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/iza-mesc_150kb_420x3/PI2/PI2_significant_95_chr12_iza-mesc_150kb_420x3.txt"))
chr12_dopa_150kb_sign_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/dopa30_150kb_482x3/PI2/PI2_significant_95_chr12_dopa30_150kb_482x3.txt"))

chr18_1NP_150kb_sign_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/mesc_46C_150kb_481x1/PI2/PI2_significant_95_chr18_mesc_46C_150kb_481x1.txt"))
chr18_iza_150kb_sign_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/iza-mesc_150kb_420x3/PI2/PI2_significant_95_chr18_iza-mesc_150kb_420x3.txt"))
chr18_dopa_150kb_sign_pi = squareform(np.loadtxt("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/dopa30_150kb_482x3/PI2/PI2_significant_95_chr18_dopa30_150kb_482x3.txt"))

for i in [chr12_1NP_150kb_pi, chr12_iza_150kb_pi, chr12_dopa_150kb_pi, chr18_1NP_150kb_pi, chr18_iza_150kb_pi, chr18_dopa_150kb_pi, chr12_1NP_150kb_sign_pi, chr12_iza_150kb_sign_pi, chr12_dopa_150kb_sign_pi, chr18_1NP_150kb_sign_pi,
    chr18_iza_150kb_sign_pi, chr18_dopa_150kb_sign_pi]:
    np.fill_diagonal(i, np.nan)

In [8]:
chr12_18_1NP_150kb_pi  = np.load("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/mesc_46C_150kb_481x1/PI2_inter_beta_evaluation/PI2_inter_chr12_chr18_mesc_46C_150kb_481x1.npy")
chr12_18_iza_150kb_pi  = np.load("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/iza-mesc_150kb_420x3/PI2_inter_beta_evaluation/PI2_inter_chr12_chr18_iza-mesc_150kb_420x3.npy")
chr12_18_dopa_150kb_pi = np.load("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/dopa30_150kb_482x3/PI2_inter_beta_evaluation/PI2_inter_chr12_chr18_dopa30_150kb_482x3.npy")

chr12_18_1NP_150kb_sign_pi  = np.load("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/mesc_46C_150kb_481x1/PI2_inter_beta_evaluation/PI2_inter_significant_95_chr12_chr18_mesc_46C_150kb_481x1.npy")
chr12_18_iza_150kb_sign_pi  = np.load("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/iza-mesc_150kb_420x3/PI2_inter_beta_evaluation/PI2_inter_significant_95_chr12_chr18_iza-mesc_150kb_420x3.npy")
chr12_18_dopa_150kb_sign_pi = np.load("/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/dopa30_150kb_482x3/PI2_inter_beta_evaluation/PI2_inter_significant_95_chr12_chr18_dopa30_150kb_482x3.npy")

# Heatmaps #

## pi ##

In [9]:
fig1, _, _ = plot_heatmap_two_chromosomes(chr12_1NP_150kb_pi, chr18_1NP_150kb_pi, chr12_18_1NP_150kb_pi, colormap = "RdYlBu_r", title = "chr 12 18 pi 1NP", resolution=150E3);
fig2, _, _ = plot_heatmap_two_chromosomes(chr12_iza_150kb_pi, chr18_iza_150kb_pi, chr12_18_iza_150kb_pi, colormap = "RdYlBu_r", title = "chr 12 18 pi iza", resolution=150E3);
fig3, _, _ = plot_heatmap_two_chromosomes(chr12_dopa_150kb_pi, chr18_dopa_150kb_pi, chr12_18_dopa_150kb_pi, colormap = "RdYlBu_r", title = "chr 12 18 pi dopa", resolution=150E3)

fig1.savefig(save_folder + "heatmaps/pi_full_chromosomes/" +  'chr12_18_1NP_150kb_pi.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "heatmaps/pi_full_chromosomes/" +  'chr12_18_iza_150kb_pi.svg' , format = 'svg', dpi = 300)
fig3.savefig(save_folder + "heatmaps/pi_full_chromosomes/" +  'chr12_18_dopa_150kb_pi.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)
plt.close(fig3)

del fig1, fig2, fig3

In [10]:
fig1, _, _ = plot_heatmap_two_chromosomes(chr12_1NP_150kb_sign_pi, chr18_1NP_150kb_sign_pi, chr12_18_1NP_150kb_sign_pi, colormap = "RdYlBu_r", title = "chr 12 18 sign pi 1NP", resolution=150E3);
fig2, _, _ = plot_heatmap_two_chromosomes(chr12_iza_150kb_sign_pi, chr18_iza_150kb_sign_pi, chr12_18_iza_150kb_sign_pi, colormap = "RdYlBu_r", title = "chr 12 18 sign pi iza", resolution=150E3);
fig3, _, _ = plot_heatmap_two_chromosomes(chr12_dopa_150kb_sign_pi, chr18_dopa_150kb_sign_pi, chr12_18_dopa_150kb_sign_pi, colormap = "RdYlBu_r", title = "chr 12 18 sign pi dopa", resolution=150E3)

fig1.savefig(save_folder + "heatmaps/sign_pi_full_chromosomes/" +  'chr12_18_1NP_150kb_sign_pi.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "heatmaps/sign_pi_full_chromosomes/" +  'chr12_18_iza_150kb_sign_pi.svg' , format = 'svg', dpi = 300)
fig3.savefig(save_folder + "heatmaps/sign_pi_full_chromosomes/" +  'chr12_18_dopa_150kb_sign_pi.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)
plt.close(fig3)

del fig1, fig2, fig3

In [11]:
fig1, _, _, _ = plot_heatmap(chr12_18_1NP_150kb_pi, x_figsize=10, y_figsize=10,colormap="RdYlBu_r", title = "chr 12 18 1NP pi ", resolution=150E3)
fig2, _, _, _ = plot_heatmap(chr12_18_iza_150kb_pi, x_figsize=10, y_figsize=10,colormap="RdYlBu_r", title = "chr 12 18 iza pi ", resolution=150E3)
fig3, _, _, _ = plot_heatmap(chr12_18_dopa_150kb_pi,x_figsize=10, y_figsize=10, colormap="RdYlBu_r", title = "chr 12 18 dopa pi ", resolution=150E3)

fig1.savefig(save_folder + "heatmaps/inter_pi/" +  'chr12_18_1NP_150kb_pi.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "heatmaps/inter_pi/" +  'chr12_18_iza_150kb_pi.svg' , format = 'svg', dpi = 300)
fig3.savefig(save_folder + "heatmaps/inter_pi/" +  'chr12_18_dopa_150kb_pi.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)
plt.close(fig3)

del fig1, fig2, fig3

In [12]:
fig1, _, _, _ = plot_heatmap(chr12_18_1NP_150kb_sign_pi,x_figsize=10, y_figsize=10, colormap="RdYlBu_r", title = "chr 12 18 1NP sign pi", resolution=150E3)
fig2, _, _, _ = plot_heatmap(chr12_18_iza_150kb_sign_pi,x_figsize=10, y_figsize=10, colormap="RdYlBu_r", title = "chr 12 18 iza sign pi", resolution=150E3)
fig3, _, _, _ = plot_heatmap(chr12_18_dopa_150kb_sign_pi,x_figsize=10, y_figsize=10, colormap="RdYlBu_r", title = "chr 12 18 dopa sign pi", resolution=150E3)

fig1.savefig(save_folder + "heatmaps/inter_sign_pi/" +  'chr12_18_1NP_150kb_sign_pi.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "heatmaps/inter_sign_pi/" +  'chr12_18_iza_150kb_sign_pi.svg' , format = 'svg', dpi = 300)
fig3.savefig(save_folder + "heatmaps/inter_sign_pi/" +  'chr12_18_dopa_150kb_sign_pi.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)
plt.close(fig3)

del fig1, fig2, fig3

In [13]:
fig1, _, _, _ = plot_heatmap(chr12_1NP_150kb_pi, x_figsize=10, y_figsize=10, title = "chr12_1NP_150kb_pi",colormap="RdYlBu_r", resolution = 150E3);
fig2, _, _, _ = plot_heatmap(chr12_iza_150kb_pi, x_figsize=10, y_figsize=10, title = "chr12_iza_150kb_pi" , colormap="RdYlBu_r", resolution = 150E3);
fig3, _, _, _ = plot_heatmap(chr12_dopa_150kb_pi, x_figsize=10, y_figsize=10, title = "chr12_dopa_150kb_pi" , colormap="RdYlBu_r", resolution = 150E3);

fig4, _, _, _ = plot_heatmap(chr12_1NP_150kb_sign_pi, x_figsize=10, y_figsize=10, title = "chr12_1NP_150kb_sign_pi" , colormap="RdYlBu_r", resolution = 150E3);
fig5, _, _, _ = plot_heatmap(chr12_iza_150kb_sign_pi, x_figsize=10, y_figsize=10, title = "chr12_iza_150kb_sign_pi" , colormap="RdYlBu_r", resolution = 150E3);
fig6, _, _, _ = plot_heatmap(chr12_dopa_150kb_sign_pi, x_figsize=10, y_figsize=10, title = "chr12_dopa_150kb_sign_pi" , colormap="RdYlBu_r", resolution = 150E3);

fig1.savefig(save_folder + "heatmaps/intra_pi/" +  'chr12_1NP_150kb_pi.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "heatmaps/intra_pi/" +  'chr12_iza_150kb_pi.svg' , format = 'svg', dpi = 300)
fig3.savefig(save_folder + "heatmaps/intra_pi/" +  'chr12_dopa_150kb_pi.svg' , format = 'svg', dpi = 300)

fig4.savefig(save_folder + "heatmaps/intra_sign_pi/" +  'chr12_1NP_150kb_sign_pi.svg' , format = 'svg', dpi = 300)
fig5.savefig(save_folder + "heatmaps/intra_sign_pi/" +  'chr12_iza_150kb_sign_pi.svg' , format = 'svg', dpi = 300)
fig6.savefig(save_folder + "heatmaps/intra_sign_pi/" +  'chr12_dopa_150kb_sign_pi.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)
plt.close(fig3)
plt.close(fig4)
plt.close(fig5)
plt.close(fig6)

del fig1, fig2, fig3, fig4, fig5, fig6


## Coseg ##

In [14]:
fig1, _, _ = plot_heatmap_two_chromosomes(chr12_1NP_150kb_coseg, chr18_1NP_150kb_coseg, chr12_18_1NP_150kb_coseg, colormap = "RdYlBu_r", title = "chr 12 18 coseg 1NP", resolution=150E3);
fig2, _, _ = plot_heatmap_two_chromosomes(chr12_iza_150kb_coseg, chr18_iza_150kb_coseg, chr12_18_iza_150kb_coseg, colormap = "RdYlBu_r", title = "chr 12 18 coseg iza", resolution=150E3);
fig3, _, _ = plot_heatmap_two_chromosomes(chr12_dopa_150kb_coseg, chr18_dopa_150kb_coseg, chr12_18_dopa_150kb_coseg, colormap = "RdYlBu_r", title = "chr 12 18 coseg dopa", resolution=150E3)

fig1.savefig(save_folder + "heatmaps/coseg_full_chromosomes/" +  'chr12_18_1NP_150kb_coseg.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "heatmaps/coseg_full_chromosomes/" +  'chr12_18_iza_150kb_coseg.svg' , format = 'svg', dpi = 300)
fig3.savefig(save_folder + "heatmaps/coseg_full_chromosomes/" +  'chr12_18_dopa_150kb_coseg.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)
plt.close(fig3)

del fig1, fig2, fig3

## NPMI ##

In [15]:
with warnings.catch_warnings():

    warnings.filterwarnings("ignore", "divide by zero encountered in log", category=RuntimeWarning)   
    warnings.filterwarnings("ignore", "invalid value encountered in true_divide", category=RuntimeWarning)   

    chr12_1NP_150kb_npmi = compute_npmi(segregation_table_1NP_150kb['chr12']['segregation_table'])
    chr18_1NP_150kb_npmi = compute_npmi(segregation_table_1NP_150kb['chr18']['segregation_table'])
    chr12_18_1NP_150kb_npmi = compute_npmi_inter(segregation_table_1NP_150kb['chr12']['segregation_table'], segregation_table_1NP_150kb['chr18']['segregation_table'])


    fig1, _, _ = plot_heatmap_two_chromosomes(chr12_1NP_150kb_npmi, chr18_1NP_150kb_npmi, chr12_18_1NP_150kb_npmi, colormap = "RdYlBu_r", title = "chr 12 18 NPMI 1NP", resolution=150E3);

    chr12_iza_150kb_npmi = compute_npmi(segregation_table_iza_150kb['chr12']['segregation_table'])
    chr18_iza_150kb_npmi = compute_npmi(segregation_table_iza_150kb['chr18']['segregation_table'])
    chr12_18_iza_150kb_npmi = compute_npmi_inter(segregation_table_iza_150kb['chr12']['segregation_table'], segregation_table_iza_150kb['chr18']['segregation_table'])


    fig2, _, _ = plot_heatmap_two_chromosomes(chr12_iza_150kb_npmi, chr18_iza_150kb_npmi, chr12_18_iza_150kb_npmi, colormap = "RdYlBu_r", title = "chr 12 18 NPMI iza", resolution=150E3);

    chr12_dopa_150kb_npmi = compute_npmi(segregation_table_dopa_150kb['chr12']['segregation_table'])
    chr18_dopa_150kb_npmi = compute_npmi(segregation_table_dopa_150kb['chr18']['segregation_table'])
    chr12_18_dopa_150kb_npmi = compute_npmi_inter(segregation_table_dopa_150kb['chr12']['segregation_table'], segregation_table_dopa_150kb['chr18']['segregation_table'])


    fig3, _, _ = plot_heatmap_two_chromosomes(chr12_dopa_150kb_npmi, chr18_dopa_150kb_npmi, chr12_18_dopa_150kb_npmi, colormap = "RdYlBu_r", title = "chr 12 18 NPMI dopa", resolution=150E3)

fig1.savefig(save_folder + "heatmaps/NPMI_full_chromosomes/" +  'chr12_18_1NP_150kb_npmi.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "heatmaps/NPMI_full_chromosomes/" +  'chr12_18_iza_150kb_npmi.svg' , format = 'svg', dpi = 300)
fig3.savefig(save_folder + "heatmaps/NPMI_full_chromosomes/" +  'chr12_18_dopa_150kb_npmi.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)
plt.close(fig3)

del fig1, fig2, fig3

In [16]:
fig1, _, _, _ = plot_heatmap(chr12_18_1NP_150kb_npmi, colormap="RdYlBu_r", title="chr12_18_1NP_150kb_npmi");
fig2, _, _, _ = plot_heatmap(chr12_18_iza_150kb_npmi, colormap="RdYlBu_r", title="chr12_18_iza_150kb_npmi");
fig3, _, _, _ = plot_heatmap(chr12_18_dopa_150kb_npmi, colormap="RdYlBu_r", title="chr12_18_dopa_150kb_npmi");

fig1.savefig(save_folder + "heatmaps/inter_NPMI/" +  'chr12_18_1NP_150kb_npmi_inter.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "heatmaps/inter_NPMI/" +  'chr12_18_iza_150kb_npmi_inter.svg' , format = 'svg', dpi = 300)
fig3.savefig(save_folder + "heatmaps/inter_NPMI/" +  'chr12_18_dopa_150kb_npmi_inter.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)
plt.close(fig3)

del fig1, fig2, fig3


# WDF #

In [17]:
chr12_start, chr12_end, chr18_start, chr18_end = select_chromosomes_coordinates(7, 18, chr_windows_sizes)

In [18]:
chr12_18_1NP_150kb_wdf = np.array([])
chr12_18_iza_150kb_wdf = np.array([])
chr12_18_dopa_150kb_wdf = np.array([])

chr_list = ["chr12","chr18"]

for chr in chr_list:
    F_arr = compute_tube_segregation_frequency(segregation_table_1NP_150kb[chr]["segregation_table"])
    chr12_18_1NP_150kb_wdf = np.concatenate((chr12_18_1NP_150kb_wdf, F_arr))
    del F_arr

    F_arr = compute_tube_segregation_frequency(segregation_table_iza_150kb[chr]["segregation_table"])
    chr12_18_iza_150kb_wdf = np.concatenate((chr12_18_iza_150kb_wdf, F_arr))
    del F_arr

    F_arr = compute_tube_segregation_frequency(segregation_table_dopa_150kb[chr]["segregation_table"])
    chr12_18_dopa_150kb_wdf = np.concatenate((chr12_18_dopa_150kb_wdf, F_arr))
    del F_arr

In [19]:
fig1, ax1 = plt.subplots(1, figsize = (10, 1))
fig2, ax2 = plt.subplots(1, figsize = (10, 1))
fig3, ax3 = plt.subplots(1, figsize = (10, 1))

ax1.plot(chr12_18_1NP_150kb_wdf,  linewidth = 0.5, color = "orange");     ax1.axvline(x= 0, color = "black");  ax1.axvline(x=chr12_end - chr12_start, color = "black");  ax1.axvline(x=chr12_end - chr12_start + chr18_end - chr18_start, color = "black");  ax1.set_title("chr12_18_1NP_150kb_wdf");
ax2.plot(chr12_18_iza_150kb_wdf,  linewidth = 0.5, color = "orange");     ax2.axvline(x= 0, color = "black");  ax2.axvline(x=chr12_end - chr12_start, color = "black");  ax2.axvline(x=chr12_end - chr12_start + chr18_end - chr18_start, color = "black");  ax2.set_title("chr12_18_iza_150kb_wdf");
ax3.plot(chr12_18_dopa_150kb_wdf, linewidth = 0.5, color = "orange");     ax3.axvline(x= 0, color = "black");  ax3.axvline(x=chr12_end - chr12_start, color = "black");  ax3.axvline(x=chr12_end - chr12_start + chr18_end - chr18_start, color = "black");  ax3.set_title("chr12_18_dopa_150kb_wdf");

#ax1.fill_between(np.arange(0, chr12_18_1NP_150kb_wdf.shape[0], 1),  0, chr12_18_1NP_150kb_wdf, color = "orange")
#ax2.fill_between(np.arange(0, chr12_18_iza_150kb_wdf.shape[0], 1),  0, chr12_18_iza_150kb_wdf, color = "orange")
#ax3.fill_between(np.arange(0, chr12_18_dopa_150kb_wdf.shape[0], 1), 0, chr12_18_dopa_150kb_wdf, color = "orange")

fig1.savefig(save_folder + "WDF/" +  'chr12_18_1NP_150kb_wdf.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "WDF/" +  'chr12_18_iza_150kb_wdf.svg' , format = 'svg', dpi = 300)
fig3.savefig(save_folder + "WDF/" +  'chr12_18_dopa_150kb_wdf.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)
plt.close(fig3)

del fig1, fig2, fig3, ax1, ax2, ax3

In [20]:
fig1, _, _ = plot_heatmap_window_seg_freq(chr12_18_1NP_150kb_pi, chr18_WDF_1NP_150kb,      chr12_WDF_1NP_150kb, title = "chr12_18_1NP_150kb_pi_beta_eval", resolution=150E3, colormap = "RdYlBu_r")
fig2, _, _ = plot_heatmap_window_seg_freq(chr12_18_1NP_150kb_sign_pi, chr18_WDF_1NP_150kb, chr12_WDF_1NP_150kb, title = "chr12_18_1NP_150kb_sign_pi_beta_eval", resolution=150E3, colormap = "RdYlBu_r")
fig3, _, _ = plot_heatmap_window_seg_freq(chr12_18_1NP_150kb_npmi, chr18_WDF_1NP_150kb,    chr12_WDF_1NP_150kb, title = "chr12_18_1NP_150kb_npmi", resolution=150E3, colormap = "RdYlBu_r")

fig1.savefig(save_folder + 'heatmaps/inter_WDF/chr12_18_1NP_150kb_pi_WDF_colormap2.png', format = 'png')
fig2.savefig(save_folder + 'heatmaps/inter_WDF/chr12_18_1NP_150kb_sign_pi_WDF_colormap2.png', format = 'png')
fig3.savefig(save_folder + 'heatmaps/inter_WDF/chr12_18_1NP_150kb_npmi.png', format = 'png')

plt.close(fig1);    plt.close(fig2);    plt.close(fig3);    

del fig1, fig2, fig3

# Histograms #

In [21]:
fig1, ax1 = plt.subplots(1, figsize = (10,10))
fig2, ax2 = plt.subplots(1, figsize = (10,10))
fig3, ax3 = plt.subplots(1, figsize = (10,10))
fig4, ax4 = plt.subplots(1, figsize = (10,10))
fig5, ax5 = plt.subplots(1, figsize = (10,10))
fig6, ax6 = plt.subplots(1, figsize = (10,10))

ax1.hist(chr12_1NP_150kb_pi.flatten(), bins = np.linspace(0,np.nanmax(chr12_1NP_150kb_pi), 50), density = True, edgecolor = 'white');
ax1.set_title("chr12_1NP_150kb_pi\n");

ax2.hist(chr12_1NP_150kb_sign_pi.flatten(), bins = np.linspace(0,np.nanmax(chr12_1NP_150kb_sign_pi), 50), density = True, edgecolor = 'white');
ax2.set_title("chr12_1NP_150kb_sign_pi\n");

ax3.hist(chr18_1NP_150kb_pi.flatten(), bins = np.linspace(0,np.nanmax(chr18_1NP_150kb_pi), 50), density = True, edgecolor = 'white');
ax3.set_title("chr18_1NP_150kb_pi\n");

ax4.hist(chr18_1NP_150kb_sign_pi.flatten(), bins = np.linspace(0,np.nanmax(chr18_1NP_150kb_sign_pi), 50), density = True, edgecolor = 'white');
ax4.set_title("chr18_1NP_150kb_sign_pi\n");

ax5.hist(chr12_18_1NP_150kb_pi.flatten(), bins = np.linspace(0,np.nanmax(chr12_18_1NP_150kb_pi), 50), edgecolor = "white", density = True);
ax5.set_title("chr12_18_1NP_150kb_pi\n")

ax6.hist(chr12_18_1NP_150kb_sign_pi.flatten(), bins = np.linspace(0,np.nanmax(chr12_18_1NP_150kb_sign_pi), 50), edgecolor = "white", density = True);
ax6.set_title("chr12_18_1NP_150kb_sign_pi\n")

ax1.tick_params(axis='x', which='major', labelsize=30); ax1.tick_params(axis='y', which='major', labelsize=30)
ax2.tick_params(axis='x', which='major', labelsize=30); ax2.tick_params(axis='y', which='major', labelsize=30)
ax3.tick_params(axis='x', which='major', labelsize=30); ax3.tick_params(axis='y', which='major', labelsize=30)
ax4.tick_params(axis='x', which='major', labelsize=30); ax4.tick_params(axis='y', which='major', labelsize=30)
ax5.tick_params(axis='x', which='major', labelsize=30); ax5.tick_params(axis='y', which='major', labelsize=30)
ax6.tick_params(axis='x', which='major', labelsize=30); ax6.tick_params(axis='y', which='major', labelsize=30)

fig1.savefig(save_folder + 'histograms/beagrie_mesc_1NP/chr12_1NP_150kb_pi_hist.svg', format = 'svg')
fig2.savefig(save_folder + 'histograms/beagrie_mesc_1NP/chr12_1NP_150kb_sign_pi_hist.svg', format = 'svg')
fig3.savefig(save_folder + 'histograms/beagrie_mesc_1NP/chr18_1NP_150kb_pi_hist.svg', format = 'svg')
fig4.savefig(save_folder + 'histograms/beagrie_mesc_1NP/chr18_1NP_150kb_sign_pi_hist.svg', format = 'svg')
fig5.savefig(save_folder + 'histograms/beagrie_mesc_1NP/chr12_18_1NP_150kb_pi_hist.svg', format = 'svg')
fig6.savefig(save_folder + 'histograms/beagrie_mesc_1NP/chr12_18_1NP_150kb_sign_pi_hist.svg', format = 'svg')

fig1.savefig(save_folder + 'histograms/beagrie_mesc_1NP/chr12_1NP_150kb_pi_hist.png', format = 'png')
fig2.savefig(save_folder + 'histograms/beagrie_mesc_1NP/chr12_1NP_150kb_sign_pi_hist.png', format = 'png')
fig3.savefig(save_folder + 'histograms/beagrie_mesc_1NP/chr18_1NP_150kb_pi_hist.png', format = 'png')
fig4.savefig(save_folder + 'histograms/beagrie_mesc_1NP/chr18_1NP_150kb_sign_pi_hist.png', format = 'png')
fig5.savefig(save_folder + 'histograms/beagrie_mesc_1NP/chr12_18_1NP_150kb_pi_hist.png', format = 'png')
fig6.savefig(save_folder + 'histograms/beagrie_mesc_1NP/chr12_18_1NP_150kb_sign_pi_hist.png', format = 'png')

plt.close(fig1);    plt.close(fig2);    plt.close(fig3);    plt.close(fig4);    plt.close(fig5);    plt.close(fig6);    

del fig1, ax1, fig2, ax2, fig3, ax3, fig4, ax4, fig5, ax5, fig6, ax6

# Stats #

In [22]:
chr12_1NP_stats = compute_pi_nan(chr12_1NP_150kb_pi, chr12_1NP_150kb_sign_pi, segregation_table_1NP_150kb['chr12']['segregation_table'])
chr18_1NP_stats = compute_pi_nan(chr18_1NP_150kb_pi, chr18_1NP_150kb_sign_pi, segregation_table_1NP_150kb['chr18']['segregation_table']);
inter_12_18_1NP_stats = compute_pi_nan_inter(chr12_18_1NP_150kb_pi, chr12_18_1NP_150kb_sign_pi, segregation_table_1NP_150kb['chr12']['segregation_table'], segregation_table_1NP_150kb['chr18']['segregation_table']);

header = "chr\tmean_pi\tstd_pi\tmean_sign_pi\tstd_sign_pi\tGAM_nan\tpi<0\tpercent_sign_pi\tpi>1"
chr12_info = "chr12\t" + str(np.nanmean(chr12_1NP_150kb_pi)) + "\t" + str(np.nanstd(chr12_1NP_150kb_pi)) + "\t" + str(np.nanmean(chr12_1NP_150kb_sign_pi)) + "\t" + str(np.nanstd(chr12_1NP_150kb_sign_pi)) + "\t" + str(chr12_1NP_stats[0]) + "\t" + str(chr12_1NP_stats[1]) + "\t" + str(chr12_1NP_stats[2]) + "\t" + str(chr12_1NP_stats[3])
chr18_info = "chr18\t" + str(np.nanmean(chr18_1NP_150kb_pi)) + "\t" + str(np.nanstd(chr18_1NP_150kb_pi)) + "\t" + str(np.nanmean(chr18_1NP_150kb_sign_pi)) + "\t" + str(np.nanstd(chr18_1NP_150kb_sign_pi)) + "\t" + str(chr18_1NP_stats[0]) + "\t" + str(chr18_1NP_stats[1]) + "\t" + str(chr18_1NP_stats[2]) + "\t" + str(chr18_1NP_stats[3])
inter_info = "chr12_18\t" + str(np.nanmean(chr12_18_1NP_150kb_pi)) + "\t" + str(np.nanstd(chr12_18_1NP_150kb_pi)) + "\t" + str(np.nanmean(chr12_18_1NP_150kb_sign_pi)) + "\t" + str(np.nanstd(chr12_18_1NP_150kb_sign_pi)) + "\t" + str(inter_12_18_1NP_stats[0]) + "\t" + str(inter_12_18_1NP_stats[1]) + "\t" + str(inter_12_18_1NP_stats[2]) + "\t" + str(inter_12_18_1NP_stats[3])

with open(save_folder + "tables/" + 'beagrie_mesc_1NP_stats.txt', 'w') as f:
    f.write(header + "\n" + chr12_info + "\n"  + chr18_info + "\n"  + inter_info)

chr12_iza_stats = compute_pi_nan(chr12_iza_150kb_pi, chr12_iza_150kb_sign_pi, segregation_table_iza_150kb['chr12']['segregation_table'])
chr18_iza_stats = compute_pi_nan(chr18_iza_150kb_pi, chr18_iza_150kb_sign_pi, segregation_table_iza_150kb['chr18']['segregation_table']);
inter_12_18_iza_stats = compute_pi_nan_inter(chr12_18_iza_150kb_pi, chr12_18_iza_150kb_sign_pi, segregation_table_iza_150kb['chr12']['segregation_table'], segregation_table_iza_150kb['chr18']['segregation_table']);


chr12_info = "chr12\t" + str(np.nanmean(chr12_iza_150kb_pi)) + "\t" + str(np.nanstd(chr12_iza_150kb_pi)) + "\t" + str(np.nanmean(chr12_iza_150kb_sign_pi)) + "\t" + str(np.nanstd(chr12_iza_150kb_sign_pi)) + "\t" + str(chr12_iza_stats[0]) + "\t" + str(chr12_iza_stats[1]) + "\t" + str(chr12_iza_stats[2]) + "\t" + str(chr12_iza_stats[3])
chr18_info = "chr18\t" + str(np.nanmean(chr18_iza_150kb_pi)) + "\t" + str(np.nanstd(chr18_iza_150kb_pi)) + "\t" + str(np.nanmean(chr18_iza_150kb_sign_pi)) + "\t" + str(np.nanstd(chr18_iza_150kb_sign_pi)) + "\t" + str(chr18_iza_stats[0]) + "\t" + str(chr18_iza_stats[1]) + "\t" + str(chr18_iza_stats[2]) + "\t" + str(chr18_iza_stats[3])
inter_info = "chr12_18\t" + str(np.nanmean(chr12_18_iza_150kb_pi)) + "\t" + str(np.nanstd(chr12_18_iza_150kb_pi)) + "\t" + str(np.nanmean(chr12_18_iza_150kb_sign_pi)) + "\t" + str(np.nanstd(chr12_18_iza_150kb_sign_pi)) + "\t" + str(inter_12_18_iza_stats[0]) + "\t" + str(inter_12_18_iza_stats[1]) + "\t" + str(inter_12_18_iza_stats[2]) + "\t" + str(inter_12_18_iza_stats[3])

with open(save_folder  + "tables/"+ 'iza_mesc_3NP_stats.txt', 'w') as f:
    f.write(header + "\n" + chr12_info + "\n"  + chr18_info + "\n"  + inter_info)

chr12_dopa_stats = compute_pi_nan(chr12_dopa_150kb_pi, chr12_dopa_150kb_sign_pi, segregation_table_dopa_150kb['chr12']['segregation_table'])
chr18_dopa_stats = compute_pi_nan(chr18_dopa_150kb_pi, chr18_dopa_150kb_sign_pi, segregation_table_dopa_150kb['chr18']['segregation_table']);
inter_12_18_dopa_stats = compute_pi_nan_inter(chr12_18_dopa_150kb_pi, chr12_18_dopa_150kb_sign_pi, segregation_table_dopa_150kb['chr12']['segregation_table'], segregation_table_dopa_150kb['chr18']['segregation_table']);



chr12_info = "chr12\t" + str(np.nanmean(chr12_dopa_150kb_pi)) + "\t" + str(np.nanstd(chr12_dopa_150kb_pi)) + "\t" + str(np.nanmean(chr12_dopa_150kb_sign_pi)) + "\t" + str(np.nanstd(chr12_dopa_150kb_sign_pi)) + "\t" + str(chr12_dopa_stats[0]) + "\t" + str(chr12_dopa_stats[1]) + "\t" + str(chr12_dopa_stats[2]) + "\t" + str(chr12_dopa_stats[3])
chr18_info = "chr18\t" + str(np.nanmean(chr18_dopa_150kb_pi)) + "\t" + str(np.nanstd(chr18_dopa_150kb_pi)) + "\t" + str(np.nanmean(chr18_dopa_150kb_sign_pi)) + "\t" + str(np.nanstd(chr18_dopa_150kb_sign_pi)) + "\t" + str(chr18_dopa_stats[0]) + "\t" + str(chr18_dopa_stats[1]) + "\t" + str(chr18_dopa_stats[2]) + "\t" + str(chr18_dopa_stats[3])
inter_info = "chr12_18\t" + str(np.nanmean(chr12_18_dopa_150kb_pi)) + "\t" + str(np.nanstd(chr12_18_dopa_150kb_pi)) + "\t" + str(np.nanmean(chr12_18_dopa_150kb_sign_pi)) + "\t" + str(np.nanstd(chr12_18_dopa_150kb_sign_pi)) + "\t" + str(inter_12_18_dopa_stats[0]) + "\t" + str(inter_12_18_dopa_stats[1]) + "\t" + str(inter_12_18_dopa_stats[2]) + "\t" + str(inter_12_18_dopa_stats[3])

with open(save_folder + "tables/" + 'dopa_3NP_stats.txt', 'w') as f:
    f.write(header + "\n" + chr12_info + "\n"  + chr18_info + "\n"  + inter_info)

# chr12:1-60Mb chr18:1-60Mb #

In [23]:
chr12_detail_start = 0
chr12_detail_end   = 400

chr18_detail_start = 0
chr18_detail_end   = 400

## NPMI ##

In [24]:
fig1, _, _ = plot_heatmap_two_chromosomes(chr12_1NP_150kb_npmi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], chr18_1NP_150kb_npmi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], chr12_18_1NP_150kb_npmi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], colormap = "RdYlBu_r", title = "chr 12 18 NPMI 1NP", resolution=150E3)
fig2, _, _ = plot_heatmap_two_chromosomes(chr12_iza_150kb_npmi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], chr18_iza_150kb_npmi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], chr12_18_iza_150kb_npmi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], colormap = "RdYlBu_r", title = "chr 12 18 NPMI iza", resolution=150E3)
fig3, _, _ = plot_heatmap_two_chromosomes(chr12_dopa_150kb_npmi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], chr18_dopa_150kb_npmi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], chr12_18_dopa_150kb_npmi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], colormap = "RdYlBu_r", title = "chr 12 18 NPMI dopa", resolution=150E3)

fig1.savefig(save_folder + "detail/NPMI/" +  'chr12_18_1NP_150kb_npmi_detail.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "detail/NPMI/" +  'chr12_18_iza_150kb_npmi_detail.svg' , format = 'svg', dpi = 300)
fig3.savefig(save_folder + "detail/NPMI/" +  'chr12_18_dopa_150kb_npmi_detail.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)
plt.close(fig3)

del fig1
del fig2
del fig3

## $\pi$ ##

In [25]:
fig1, _, _ = plot_heatmap_two_chromosomes(chr12_1NP_150kb_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], chr18_1NP_150kb_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], chr12_18_1NP_150kb_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], colormap = "RdYlBu_r", title = "chr 12 18 pi 1NP", resolution=150E3)
fig2, _, _ = plot_heatmap_two_chromosomes(chr12_iza_150kb_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], chr18_iza_150kb_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], chr12_18_iza_150kb_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], colormap = "RdYlBu_r", title = "chr 12 18 pi iza", resolution=150E3)
fig3, _, _ = plot_heatmap_two_chromosomes(chr12_dopa_150kb_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], chr18_dopa_150kb_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], chr12_18_dopa_150kb_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], colormap = "RdYlBu_r", title = "chr 12 18 dopa", resolution=150E3)

fig1.savefig(save_folder + "detail/pi/" +  'chr12_18_1NP_150kb_pi_detail.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "detail/pi/" +  'chr12_18_iza_150kb_pi_detail.svg' , format = 'svg', dpi = 300)
fig3.savefig(save_folder + "detail/pi/" +  'chr12_18_dopa_150kb_pi_detail.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)
plt.close(fig3)

del fig1
del fig2
del fig3

## $\pi$ inter ##

In [26]:
fig1, _, _, _ = plot_heatmap(chr12_18_1NP_150kb_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], colormap = "RdYlBu_r", title = "chr 12 18 pi 1NP")
fig2, _, _, _ = plot_heatmap(chr12_18_iza_150kb_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], colormap = "RdYlBu_r", title = "chr 12 18 pi iza")
fig3, _, _, _ = plot_heatmap(chr12_18_dopa_150kb_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], colormap = "RdYlBu_r", title = "chr 12 18 dopa")

fig1.savefig(save_folder + "detail/inter_pi/" +  'chr12_18_1NP_150kb_pi_detail.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "detail/inter_pi/" +  'chr12_18_iza_150kb_pi_detail.svg' , format = 'svg', dpi = 300)
fig3.savefig(save_folder + "detail/inter_pi/" +  'chr12_18_dopa_150kb_pi_detail.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)
plt.close(fig3)

del fig1
del fig2
del fig3

## $\pi$ significant inter ##

In [27]:
fig1, _, _, _ = plot_heatmap(chr12_18_1NP_150kb_sign_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], colormap = "RdYlBu_r", title = "chr 12 18 sign pi 1NP")
fig2, _, _, _ = plot_heatmap(chr12_18_iza_150kb_sign_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], colormap = "RdYlBu_r", title = "chr 12 18 sign pi iza")
fig3, _, _, _ = plot_heatmap(chr12_18_dopa_150kb_sign_pi[chr12_detail_start:chr12_detail_end, chr18_detail_start:chr18_detail_end], colormap = "RdYlBu_r", title = "chr 12 18 sign pi dopa")

fig1.savefig(save_folder + "detail/inter_pi_sign/" +  'chr12_18_1NP_150kb_sign_pi_detail.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "detail/inter_pi_sign/" +  'chr12_18_iza_150kb_sign_pi_detail.svg' , format = 'svg', dpi = 300)
fig3.savefig(save_folder + "detail/inter_pi_sign/" +  'chr12_18_dopa_150kb_sign_pi_detail.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)
plt.close(fig3)

del fig1
del fig2
del fig3

## wdf ##

In [28]:
chr12_1NP_150kb_wdf = compute_tube_segregation_frequency(segregation_table_1NP_150kb["chr12"]["segregation_table"])
chr18_1NP_150kb_wdf = compute_tube_segregation_frequency(segregation_table_1NP_150kb["chr18"]["segregation_table"])

fig1, ax1 = plt.subplots(1, figsize = (10, 1))
fig2, ax2 = plt.subplots(1, figsize = (10, 1))

ax1.plot(chr12_1NP_150kb_wdf[chr12_detail_start:chr12_detail_end],  linewidth = 0.5, color = "orange");     ax1.axvline(x= 0, color = "black");  ax1.axvline(x=400, color = "black");  ax1.set_title("chr12_1NP_150kb_wdf");
ax2.plot(chr18_1NP_150kb_wdf[chr18_detail_start:chr18_detail_end],  linewidth = 0.5, color = "orange");     ax2.axvline(x= 0, color = "black");  ax2.axvline(x=400, color = "black");  ax2.set_title("chr18_1NP_150kb_wdf");

fig1.savefig(save_folder + "detail/WDF/" +  'chr12_1NP_150kb_wdf.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "detail/WDF/" +  'chr18_1NP_150kb_wdf.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)

del fig1, fig2, ax1, ax2

In [29]:
chr12_dopa_150kb_wdf = compute_tube_segregation_frequency(segregation_table_dopa_150kb["chr12"]["segregation_table"])
chr18_dopa_150kb_wdf = compute_tube_segregation_frequency(segregation_table_dopa_150kb["chr18"]["segregation_table"])

fig1, ax1 = plt.subplots(1, figsize = (10, 1))
fig2, ax2 = plt.subplots(1, figsize = (10, 1))

ax1.plot(chr12_dopa_150kb_wdf[chr12_detail_start:chr12_detail_end],  linewidth = 0.5, color = "orange");     ax1.axvline(x= 0, color = "black");  ax1.axvline(x=400, color = "black");  ax1.set_title("chr12_dopa_150kb_wdf");
ax2.plot(chr18_dopa_150kb_wdf[chr18_detail_start:chr18_detail_end],  linewidth = 0.5, color = "orange");     ax2.axvline(x= 0, color = "black");  ax2.axvline(x=400, color = "black");  ax2.set_title("chr18_dopa_150kb_wdf");

fig1.savefig(save_folder + "detail/WDF/" +  'chr12_dopa_150kb_wdf.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + "detail/WDF/" +  'chr18_dopa_150kb_wdf.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)

del fig1, fig2, ax1, ax2

# Correlations #

In [30]:
coseg_vs_NPMI_chr12 = matrix_correlation(chr12_1NP_150kb_coseg, chr12_1NP_150kb_npmi, verbose_print=True)
fig1, ax1 = scatter_matrices(chr12_1NP_150kb_coseg, chr12_1NP_150kb_npmi, x_label="Coseg", y_label="npmi", title="chr12 mESC 1NP 150kb")

coseg_vs_NPMI_chr18 = matrix_correlation(chr18_1NP_150kb_coseg, chr18_1NP_150kb_npmi, verbose_print=True)
fig2, ax2 = scatter_matrices(chr18_1NP_150kb_coseg, chr18_1NP_150kb_npmi, x_label="Coseg", y_label="npmi", title="chr18 mESC 1NP 150kb")

coseg_vs_NPMI_inter = matrix_correlation(chr12_18_1NP_150kb_coseg, chr12_18_1NP_150kb_npmi, verbose_print=True)
fig3, ax3 = scatter_matrices(chr12_18_1NP_150kb_coseg, chr12_18_1NP_150kb_npmi, x_label="Coseg", y_label="npmi", title="chr12  18 inter mESC 1NP 150kb")

coseg_vs_pi_chr12 = matrix_correlation(chr12_1NP_150kb_coseg, chr12_1NP_150kb_pi, verbose_print=True)
fig4, ax4 = scatter_matrices(chr12_1NP_150kb_coseg, chr12_1NP_150kb_pi, x_label="Coseg", y_label="pi", title="chr12 mESC 1NP 150kb")

coseg_vs_pi_chr18 = matrix_correlation(chr18_1NP_150kb_coseg, chr18_1NP_150kb_pi, verbose_print=True)
fig5, ax5 = scatter_matrices(chr18_1NP_150kb_coseg, chr18_1NP_150kb_pi, x_label="Coseg", y_label="pi", title="chr18 mESC 1NP 150kb")

coseg_vs_pi_inter = matrix_correlation(chr12_18_1NP_150kb_coseg, chr12_18_1NP_150kb_pi, verbose_print=True)
fig6, ax6 = scatter_matrices(chr12_18_1NP_150kb_coseg, chr12_18_1NP_150kb_pi, x_label="Coseg", y_label="pi", title="chr12  18 inter mESC 1NP 150kb")

coseg_vs_sign_pi_chr12 = matrix_correlation(chr12_1NP_150kb_coseg, chr12_1NP_150kb_sign_pi, verbose_print=True)
fig7, ax7 = scatter_matrices(chr12_1NP_150kb_coseg, chr12_1NP_150kb_sign_pi, x_label="Coseg", y_label="sign pi", title="chr12 mESC 1NP 150kb")

coseg_vs_sign_pi_chr18 = matrix_correlation(chr18_1NP_150kb_coseg, chr18_1NP_150kb_sign_pi, verbose_print=True)
fig8, ax8 = scatter_matrices(chr18_1NP_150kb_coseg, chr18_1NP_150kb_sign_pi, x_label="Coseg", y_label="sign pi", title="chr18 mESC 1NP 150kb")

coseg_vs_sign_pi_inter = matrix_correlation(chr12_18_1NP_150kb_coseg, chr12_18_1NP_150kb_sign_pi, verbose_print=True)
fig9, ax9 = scatter_matrices(chr12_18_1NP_150kb_coseg, chr12_18_1NP_150kb_sign_pi, x_label="Coseg", y_label="sign pi", title="chr12  18 inter mESC 1NP 150kb")

fig1.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr12_mESC_1NP_150kb_coseg_vs_npmi.png'  , format = 'png', dpi = 300)
fig2.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr18_mESC_1NP_150kb_coseg_vs_npmi.png'  , format = 'png', dpi = 300)
fig3.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr12_18_inter_mESC_1NP_150kb_coseg_vs_npmi.png'  , format = 'png', dpi = 300)
fig4.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr12_mESC_1NP_150kb_pi_vs_npmi.png'  , format = 'png', dpi = 300)
fig5.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr18_mESC_1NP_150kb_pi_vs_npmi.png'  , format = 'png', dpi = 300)
fig6.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr12_18_inter_mESC_1NP_150kb_pi_vs_npmi.png'  , format = 'png', dpi = 300)
fig7.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr12_mESC_1NP_150kb_sign_pi_vs_npmi.png'  , format = 'png', dpi = 300)
fig8.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr18_mESC_1NP_150kb_sign_pi_vs_npmi.png'  , format = 'png', dpi = 300)
fig9.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr12_18_inter_mESC_1NP_150kb_sign_pi_vs_npmi.png'  , format = 'png', dpi = 300)

plt.close(fig1);    plt.close(fig2);    plt.close(fig3);    plt.close(fig4);    plt.close(fig5);    plt.close(fig6);    plt.close(fig7);    plt.close(fig8);    plt.close(fig9);

del fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, fig9

pi_vs_npmi_chr12 = matrix_correlation(chr12_1NP_150kb_pi, chr12_1NP_150kb_npmi, verbose_print=True)
fig1, ax1 = scatter_matrices(chr12_1NP_150kb_pi, chr12_1NP_150kb_npmi, x_label="pi", y_label="npmi", title="chr12 mESC 1NP 150kb")

pi_vs_npmi_chr18 = matrix_correlation(chr18_1NP_150kb_pi, chr18_1NP_150kb_npmi, verbose_print=True)
fig2, ax2 = scatter_matrices(chr18_1NP_150kb_pi, chr18_1NP_150kb_npmi, x_label="pi", y_label="npmi", title="chr18 mESC 1NP 150kb")

pi_vs_npmi_inter = matrix_correlation(chr12_18_1NP_150kb_pi, chr12_18_1NP_150kb_npmi, verbose_print=True)
fig3, ax3 = scatter_matrices(chr12_18_1NP_150kb_pi, chr12_18_1NP_150kb_npmi, x_label="pi", y_label="npmi", title="chr12  18 inter mESC 1NP 150kb")

sign_pi_vs_npmi_chr12 = matrix_correlation(chr12_1NP_150kb_sign_pi, chr12_1NP_150kb_npmi, verbose_print=True)
fig4, ax4 = scatter_matrices(chr12_1NP_150kb_sign_pi, chr12_1NP_150kb_npmi, x_label="sign pi", y_label="npmi", title="chr12 mESC 1NP 150kb")

sign_pi_vs_npmi_chr18 = matrix_correlation(chr18_1NP_150kb_sign_pi, chr18_1NP_150kb_npmi, verbose_print=True)
fig5, ax5 = scatter_matrices(chr18_1NP_150kb_sign_pi, chr18_1NP_150kb_npmi, x_label="sign pi", y_label="npmi", title="chr18 mESC 1NP 150kb")

sign_pi_vs_npmi_inter = matrix_correlation(chr12_18_1NP_150kb_sign_pi, chr12_18_1NP_150kb_npmi, verbose_print=True)
fig6, ax6 = scatter_matrices(chr12_18_1NP_150kb_sign_pi, chr12_18_1NP_150kb_npmi, x_label="sign pi", y_label="npmi", title="chr12  18 inter mESC 1NP 150kb")


fig1.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr12_mESC_1NP_150kb_npmi_vs_pi.png'  , format = 'png', dpi = 300)
fig2.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr18_mESC_1NP_150kb_npmi_vs_pi.png'  , format = 'png', dpi = 300)
fig3.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr12_18_inter_mESC_1NP_150kb_npmi_vs_pi.png'  , format = 'png', dpi = 300)
fig4.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr12_mESC_1NP_150kb_npmi_vs_sign_pi.png'  , format = 'png', dpi = 300)
fig5.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr18_mESC_1NP_150kb_npmi_vs_sign_pi.png'  , format = 'png', dpi = 300)
fig6.savefig(save_folder + "matrices_scatter/beagrie_mesc_1NP/" +  'chr12_18_inter_mESC_1NP_150kb_npmi_vs_sign_pi.png'  , format = 'png', dpi = 300)

plt.close(fig1);    plt.close(fig2);    plt.close(fig3);    plt.close(fig4);    plt.close(fig5);    plt.close(fig6);

del fig1, fig2, fig3, fig4, fig5, fig6

correlations = ("coseg_vs_NPMI_chr12\t" + str(coseg_vs_NPMI_chr12[0]) + "\t" +  str(coseg_vs_NPMI_chr12[1]) + "\n" + 
    "coseg_vs_NPMI_chr18\t" + str(coseg_vs_NPMI_chr18[0]) + "\t" +  str(coseg_vs_NPMI_chr18[1]) + "\n" + 
    "coseg_vs_NPMI_inter\t" + str(coseg_vs_NPMI_inter[0]) + "\t" +  str(coseg_vs_NPMI_inter[1]) + "\n" + 
    "coseg_vs_pi_chr12\t" + str(coseg_vs_pi_chr12[0]) + "\t" +  str(coseg_vs_pi_chr12[1]) + "\n" + 
    "coseg_vs_pi_chr18\t" + str(coseg_vs_pi_chr18[0]) + "\t" +  str(coseg_vs_pi_chr18[1]) + "\n" + 
    "coseg_vs_pi_inter\t" + str(coseg_vs_pi_inter[0]) + "\t" +  str(coseg_vs_pi_inter[1]) + "\n" + 
    "coseg_vs_sign_pi_chr12\t" + str(coseg_vs_sign_pi_chr12[0]) + "\t" +  str(coseg_vs_sign_pi_chr12[1]) + "\n" + 
    "coseg_vs_sign_pi_chr18\t" + str(coseg_vs_sign_pi_chr18[0]) + "\t" +  str(coseg_vs_sign_pi_chr18[1]) + "\n" + 
    "coseg_vs_sign_pi_inter\t" + str(coseg_vs_sign_pi_inter[0]) + "\t" +  str(coseg_vs_sign_pi_inter[1]) + "\n" + 
    "pi_vs_npmi_chr12\t" + str(pi_vs_npmi_chr12[0]) + "\t" + str(pi_vs_npmi_chr12[1]) + "\n" +
    "pi_vs_npmi_chr18\t" + str(pi_vs_npmi_chr18[0]) + "\t" + str(pi_vs_npmi_chr18[1]) + "\n" +
    "pi_vs_npmi_inter\t" + str(pi_vs_npmi_inter[0]) + "\t" + str(pi_vs_npmi_inter[1]) + "\n" +
    "sign_pi_vs_npmi_chr12\t" + str(sign_pi_vs_npmi_chr12[0]) + "\t" + str(sign_pi_vs_npmi_chr12[1]) + "\n" +
    "sign_pi_vs_npmi_chr18\t" + str(sign_pi_vs_npmi_chr18[0]) + "\t" + str(sign_pi_vs_npmi_chr18[1]) + "\n" +
    "sign_pi_vs_npmi_inter\t" + str(sign_pi_vs_npmi_inter[0]) + "\t" + str(sign_pi_vs_npmi_inter[1]))

with open(save_folder + "tables/" + 'beagrie_1NP_correlations.txt', 'w') as f:
    f.write(correlations)

del correlations,coseg_vs_NPMI_chr12, coseg_vs_NPMI_chr18, coseg_vs_NPMI_inter, coseg_vs_pi_chr12, coseg_vs_pi_chr18, coseg_vs_pi_inter, coseg_vs_sign_pi_chr12, coseg_vs_sign_pi_chr18, coseg_vs_sign_pi_inter, pi_vs_npmi_chr12, pi_vs_npmi_chr18, pi_vs_npmi_inter, sign_pi_vs_npmi_chr12, sign_pi_vs_npmi_chr18, sign_pi_vs_npmi_inter

pearson			spearman
 0.8621668839476577 	 0.8300720574359631
pearson			spearman
 0.8272103024012057 	 0.771069441243428
pearson			spearman
 0.8305215505460057 	 0.8419912961723642
pearson			spearman
 0.6088988529913599 	 0.6250683251471812
pearson			spearman
 0.6014556931693138 	 0.6344305383336706
pearson			spearman
 0.8823006604990745 	 0.8643615107353008
pearson			spearman
 0.6673254020582357 	 0.6370186073686188
pearson			spearman
 0.6335048992896595 	 0.6084312050236835
pearson			spearman
 0.6955541350633093 	 0.6366231584842217
pearson			spearman
 0.5719209100356091 	 0.6076126833847579
pearson			spearman
 0.5825927458795613 	 0.6288432547095382
pearson			spearman
 0.8646982082750707 	 0.8411479452433531
pearson			spearman
 0.6928098329460413 	 0.692658191237282
pearson			spearman
 0.6790080534671142 	 0.6955352766098265
pearson			spearman
 0.7161130631800559 	 0.6455017688343312


In [31]:
coseg_vs_NPMI_chr12 = matrix_correlation(chr12_dopa_150kb_coseg, chr12_dopa_150kb_npmi, verbose_print=True)
fig1, ax1 = scatter_matrices(chr12_dopa_150kb_coseg, chr12_dopa_150kb_npmi, x_label="Coseg", y_label="npmi", title="chr12  dopa 150kb")

coseg_vs_NPMI_chr18 = matrix_correlation(chr18_dopa_150kb_coseg, chr18_dopa_150kb_npmi, verbose_print=True)
fig2, ax2 = scatter_matrices(chr18_dopa_150kb_coseg, chr18_dopa_150kb_npmi, x_label="Coseg", y_label="npmi", title="chr18  dopa 150kb")

coseg_vs_NPMI_inter = matrix_correlation(chr12_18_dopa_150kb_coseg, chr12_18_dopa_150kb_npmi, verbose_print=True)
fig3, ax3 = scatter_matrices(chr12_18_dopa_150kb_coseg, chr12_18_dopa_150kb_npmi, x_label="Coseg", y_label="npmi", title="chr12  18 inter  dopa 150kb")

coseg_vs_pi_chr12 = matrix_correlation(chr12_dopa_150kb_coseg, chr12_dopa_150kb_pi, verbose_print=True)
fig4, ax4 = scatter_matrices(chr12_dopa_150kb_coseg, chr12_dopa_150kb_pi, x_label="Coseg", y_label="pi", title="chr12  dopa 150kb")

coseg_vs_pi_chr18 = matrix_correlation(chr18_dopa_150kb_coseg, chr18_dopa_150kb_pi, verbose_print=True)
fig5, ax5 = scatter_matrices(chr18_dopa_150kb_coseg, chr18_dopa_150kb_pi, x_label="Coseg", y_label="pi", title="chr18  dopa 150kb")

coseg_vs_pi_inter = matrix_correlation(chr12_18_dopa_150kb_coseg, chr12_18_dopa_150kb_pi, verbose_print=True)
fig6, ax6 = scatter_matrices(chr12_18_dopa_150kb_coseg, chr12_18_dopa_150kb_pi, x_label="Coseg", y_label="pi", title="chr12  18 inter  dopa 150kb")

coseg_vs_sign_pi_chr12 = matrix_correlation(chr12_dopa_150kb_coseg, chr12_dopa_150kb_sign_pi, verbose_print=True)
fig7, ax7 = scatter_matrices(chr12_dopa_150kb_coseg, chr12_dopa_150kb_sign_pi, x_label="Coseg", y_label="sign pi", title="chr12  dopa 150kb")

coseg_vs_sign_pi_chr18 = matrix_correlation(chr18_dopa_150kb_coseg, chr18_dopa_150kb_sign_pi, verbose_print=True)
fig8, ax8 = scatter_matrices(chr18_dopa_150kb_coseg, chr18_dopa_150kb_sign_pi, x_label="Coseg", y_label="sign pi", title="chr18  dopa 150kb")

coseg_vs_sign_pi_inter = matrix_correlation(chr12_18_dopa_150kb_coseg, chr12_18_dopa_150kb_sign_pi, verbose_print=True)
fig9, ax9 = scatter_matrices(chr12_18_dopa_150kb_coseg, chr12_18_dopa_150kb_sign_pi, x_label="Coseg", y_label="sign pi", title="chr12  18 inter  dopa 150kb")

fig1.savefig(save_folder + "matrices_scatter/dopa/" +  'chr12_dopa_150kb_coseg_vs_npmi.png'  , format = 'png', dpi = 300)
fig2.savefig(save_folder + "matrices_scatter/dopa/" +  'chr18_dopa_150kb_coseg_vs_npmi.png'  , format = 'png', dpi = 300)
fig3.savefig(save_folder + "matrices_scatter/dopa/" +  'chr12_18_inter_150kb_coseg_vs_npmi.png'  , format = 'png', dpi = 300)
fig4.savefig(save_folder + "matrices_scatter/dopa/" +  'chr12_dopa_150kb_pi_vs_npmi.png'  , format = 'png', dpi = 300)
fig5.savefig(save_folder + "matrices_scatter/dopa/" +  'chr18_dopa_150kb_pi_vs_npmi.png'  , format = 'png', dpi = 300)
fig6.savefig(save_folder + "matrices_scatter/dopa/" +  'chr12_18_inter_dopa_150kb_pi_vs_npmi.png'  , format = 'png', dpi = 300)
fig7.savefig(save_folder + "matrices_scatter/dopa/" +  'chr12_dopa_150kb_sign_pi_vs_npmi.png'  , format = 'png', dpi = 300)
fig8.savefig(save_folder + "matrices_scatter/dopa/" +  'chr18_dopa_150kb_sign_pi_vs_npmi.png'  , format = 'png', dpi = 300)
fig9.savefig(save_folder + "matrices_scatter/dopa/" +  'chr12_18_inter_dopa_150kb_sign_pi_vs_npmi.png'  , format = 'png', dpi = 300)

plt.close(fig1);    plt.close(fig2);    plt.close(fig3);    plt.close(fig4);    plt.close(fig5);    plt.close(fig6);    plt.close(fig7);    plt.close(fig8);    plt.close(fig9);

del fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, fig9

pi_vs_npmi_chr12 = matrix_correlation(chr12_dopa_150kb_pi, chr12_dopa_150kb_npmi, verbose_print=True)
fig1, ax1 = scatter_matrices(chr12_dopa_150kb_pi, chr12_dopa_150kb_npmi, x_label="pi", y_label="npmi", title="chr12  dopa 150kb")

pi_vs_npmi_chr18 = matrix_correlation(chr18_dopa_150kb_pi, chr18_dopa_150kb_npmi, verbose_print=True)
fig2, ax2 = scatter_matrices(chr18_dopa_150kb_pi, chr18_dopa_150kb_npmi, x_label="pi", y_label="npmi", title="chr18  dopa 150kb")

pi_vs_npmi_inter = matrix_correlation(chr12_18_dopa_150kb_pi, chr12_18_dopa_150kb_npmi, verbose_print=True)
fig3, ax3 = scatter_matrices(chr12_18_dopa_150kb_pi, chr12_18_dopa_150kb_npmi, x_label="pi", y_label="npmi", title="chr12  18 inter  dopa 150kb")

sign_pi_vs_npmi_chr12 = matrix_correlation(chr12_dopa_150kb_sign_pi, chr12_dopa_150kb_npmi, verbose_print=True)
fig4, ax4 = scatter_matrices(chr12_dopa_150kb_sign_pi, chr12_dopa_150kb_npmi, x_label="sign pi", y_label="npmi", title="chr12  dopa 150kb")

sign_pi_vs_npmi_chr18 = matrix_correlation(chr18_dopa_150kb_sign_pi, chr18_dopa_150kb_npmi, verbose_print=True)
fig5, ax5 = scatter_matrices(chr18_dopa_150kb_sign_pi, chr18_dopa_150kb_npmi, x_label="sign pi", y_label="npmi", title="chr18  dopa 150kb")

sign_pi_vs_npmi_inter = matrix_correlation(chr12_18_dopa_150kb_sign_pi, chr12_18_dopa_150kb_npmi, verbose_print=True)
fig6, ax6 = scatter_matrices(chr12_18_dopa_150kb_sign_pi, chr12_18_dopa_150kb_npmi, x_label="sign pi", y_label="npmi", title="chr12  18 inter  dopa 150kb")


fig1.savefig(save_folder + "matrices_scatter/dopa/" +  'chr12_dopa_150kb_npmi_vs_pi.png'  , format = 'png', dpi = 300)
fig2.savefig(save_folder + "matrices_scatter/dopa/" +  'chr18_dopa_150kb_npmi_vs_pi.png'  , format = 'png', dpi = 300)
fig3.savefig(save_folder + "matrices_scatter/dopa/" +  'chr12_18_inter_dopa_150kb_npmi_vs_pi.png'  , format = 'png', dpi = 300)
fig4.savefig(save_folder + "matrices_scatter/dopa/" +  'chr12_dopa_150kb_npmi_vs_sign_pi.png'  , format = 'png', dpi = 300)
fig5.savefig(save_folder + "matrices_scatter/dopa/" +  'chr18_dopa_150kb_npmi_vs_sign_pi.png'  , format = 'png', dpi = 300)
fig6.savefig(save_folder + "matrices_scatter/dopa/" +  'chr12_18_inter_dopa_150kb_npmi_vs_sign_pi.png'  , format = 'png', dpi = 300)

plt.close(fig1);    plt.close(fig2);    plt.close(fig3);    plt.close(fig4);    plt.close(fig5);    plt.close(fig6);

del fig1, fig2, fig3, fig4, fig5, fig6

correlations = ("coseg_vs_NPMI_chr12\t" + str(coseg_vs_NPMI_chr12[0]) + "\t" +  str(coseg_vs_NPMI_chr12[1]) + "\n" + 
    "coseg_vs_NPMI_chr18\t" + str(coseg_vs_NPMI_chr18[0]) + "\t" +  str(coseg_vs_NPMI_chr18[1]) + "\n" + 
    "coseg_vs_NPMI_inter\t" + str(coseg_vs_NPMI_inter[0]) + "\t" +  str(coseg_vs_NPMI_inter[1]) + "\n" + 
    "coseg_vs_pi_chr12\t" + str(coseg_vs_pi_chr12[0]) + "\t" +  str(coseg_vs_pi_chr12[1]) + "\n" + 
    "coseg_vs_pi_chr18\t" + str(coseg_vs_pi_chr18[0]) + "\t" +  str(coseg_vs_pi_chr18[1]) + "\n" + 
    "coseg_vs_pi_inter\t" + str(coseg_vs_pi_inter[0]) + "\t" +  str(coseg_vs_pi_inter[1]) + "\n" + 
    "coseg_vs_sign_pi_chr12\t" + str(coseg_vs_sign_pi_chr12[0]) + "\t" +  str(coseg_vs_sign_pi_chr12[1]) + "\n" + 
    "coseg_vs_sign_pi_chr18\t" + str(coseg_vs_sign_pi_chr18[0]) + "\t" +  str(coseg_vs_sign_pi_chr18[1]) + "\n" + 
    "coseg_vs_sign_pi_inter\t" + str(coseg_vs_sign_pi_inter[0]) + "\t" +  str(coseg_vs_sign_pi_inter[1]) + "\n" + 
    "pi_vs_npmi_chr12\t" + str(pi_vs_npmi_chr12[0]) + "\t" + str(pi_vs_npmi_chr12[1]) + "\n" +
    "pi_vs_npmi_chr18\t" + str(pi_vs_npmi_chr18[0]) + "\t" + str(pi_vs_npmi_chr18[1]) + "\n" +
    "pi_vs_npmi_inter\t" + str(pi_vs_npmi_inter[0]) + "\t" + str(pi_vs_npmi_inter[1]) + "\n" +
    "sign_pi_vs_npmi_chr12\t" + str(sign_pi_vs_npmi_chr12[0]) + "\t" + str(sign_pi_vs_npmi_chr12[1]) + "\n" +
    "sign_pi_vs_npmi_chr18\t" + str(sign_pi_vs_npmi_chr18[0]) + "\t" + str(sign_pi_vs_npmi_chr18[1]) + "\n" +
    "sign_pi_vs_npmi_inter\t" + str(sign_pi_vs_npmi_inter[0]) + "\t" + str(sign_pi_vs_npmi_inter[1]))

with open(save_folder + "tables/" + 'dopa_correlations.txt', 'w') as f:
    f.write(correlations)

del correlations,coseg_vs_NPMI_chr12, coseg_vs_NPMI_chr18, coseg_vs_NPMI_inter, coseg_vs_pi_chr12, coseg_vs_pi_chr18, coseg_vs_pi_inter, coseg_vs_sign_pi_chr12, coseg_vs_sign_pi_chr18, coseg_vs_sign_pi_inter, pi_vs_npmi_chr12, pi_vs_npmi_chr18, pi_vs_npmi_inter, sign_pi_vs_npmi_chr12, sign_pi_vs_npmi_chr18, sign_pi_vs_npmi_inter

pearson			spearman
 0.7793551016189052 	 0.7010645508329373
pearson			spearman
 0.7937360810017702 	 0.7094616094006275
pearson			spearman
 0.6390175711466836 	 0.6173684487465962
pearson			spearman
 0.6847487725695467 	 0.696795670856589
pearson			spearman
 0.6703254458838248 	 0.6584225969966389
pearson			spearman
 0.8276773056032305 	 0.7948394946817439
pearson			spearman
 0.7060456689877415 	 0.6421886042507382
pearson			spearman
 0.7455470424039825 	 0.671222526949224
pearson			spearman
 0.643371252534686 	 0.5868457872675436
pearson			spearman
 0.5525733461631157 	 0.5883837509396491
pearson			spearman
 0.5641538497760976 	 0.5790894463008522
pearson			spearman
 0.7363086976069178 	 0.6934779320490781
pearson			spearman
 0.6794197563103138 	 0.6171439670892318
pearson			spearman
 0.7280375369772476 	 0.6535820972867593
pearson			spearman
 0.5973098622945439 	 0.5280762201035721
